# Python for Data Science Project Session 5: Economics and Finance

In this session, we will look at three mini projects related to Economics/Finance, applying techniques covered in Session Five and extending these a bit further.

To start we will import 

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

## Classification: 
Data set: Taiwan bankruptcy
- Pipeline
- PCA
- SMOTE
- Metrics
- KNN & Logistic regression

## Regression: 
Data set: 
- Pipelines
- Regularization
- Linear, ridge & SVC

## Unsupervised learning: 
Data set:
- Pipelines
- PCA
- Regularization
- Linear, ridge & SVC

First, lets start by reading in our data set into a pandas DataFrame and getting some info on its dimensions and composition:

From the above, we can see that our data has 